In [1]:
import os
import random
import numpy as np
from diffusers.pipelines.deprecated.spectrogram_diffusion.notes_encoder import (
    SpectrogramNotesEncoder,
)
import torch
from diffusers import MidiProcessor
from midi_player import MIDIPlayer
from midi_player.stylers import dark

torch.set_grad_enabled(False)

/home/finlay/miniconda/envs/midi-ml/lib/python3.9/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


In [2]:
device = "cuda:1"
input_path = "data/test set"
test_file = None
batch_size = 8

encoder_config = {
    "d_ff": 2048,
    "d_kv": 64,
    "d_model": 768,
    "dropout_rate": 0.1,
    "feed_forward_proj": "gated-gelu_pytorch_tanh",
    "is_decoder": False,
    "max_length": 2048,
    "num_heads": 12,
    "num_layers": 12,
    "vocab_size": 1536,
}
encoder_weights = "data/note_encoder.bin"

In [3]:
processor = MidiProcessor()
encoder = SpectrogramNotesEncoder(**encoder_config).cuda(device=device)
encoder.eval()
sd = torch.load(encoder_weights, weights_only=True)
encoder.load_state_dict(sd)

<All keys matched successfully>

In [4]:
midi_tokens = dict()

for midi_path in [
    os.path.join(input_path, file)
    for file in os.listdir(input_path)
    if file.endswith(".mid")
]:
    midi_tokens[midi_path] = processor(midi_path)  # [1, NUM_TOKENS]

keys = list(midi_tokens.keys())
all_tokens = torch.cat(
    [torch.IntTensor(midi_tokens[key][0]).view(1, -1) for key in keys]
)  # [NUM_FILES, NUM_TOKENS]

print(f"tokenized {len(midi_tokens.keys())} files")

tokenized 36 files


In [5]:
midi_embeddings = dict()
for i in range(0, all_tokens.shape[0], batch_size):
    batch = all_tokens[i : i + batch_size].cuda(
        device=device
    )  # [BATCH_SIZE, NUM_TOKENS]

    with torch.autocast("cuda"):
        tokens_mask = batch > 0  # [varies]
        tokens_embedded, tokens_mask = encoder(
            encoder_input_tokens=batch, encoder_inputs_mask=tokens_mask
        )

    for idx in range(batch.shape[0]):
        avg_embedding = tokens_embedded[idx][tokens_mask[idx]].mean(0)  # [LATENT_DIM]
        midi_embeddings[keys[i + idx]] = avg_embedding

In [6]:
if test_file is None:
    test_file = random.sample(keys, 1)[0]
    print(f"randomly selected test file {test_file}")

similarity_metric = lambda x, y: np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

test_embedding = midi_embeddings[test_file]
similarities = dict()
for file, embedding in midi_embeddings.items():
    similarities[file] = similarity_metric(
        test_embedding.cpu().detach(), embedding.cpu().detach()
    )

randomly selected test file data/test set/velocitytweaks-060-05_c4v20.mid


In [7]:
sorted_sims = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

print(f"{'Index':<10}{'Similarity':<15}{'Filename'}")
print("-" * 80)
for idx, (filename, similarity) in enumerate(sorted_sims):
    print(f"{idx:<10}{similarity:<15.4f}{filename}")

Index     Similarity     Filename
--------------------------------------------------------------------------------
0         1.0000         data/test set/velocitytweaks-060-05_c4v20.mid
1         1.0000         data/test set/velocitytweaks-060-05_c497.mid
2         1.0000         data/test set/velocitytweaks-060-05_c4vel100.mid
3         0.6352         data/test set/velocitytweaks-060-05_c3vel100.mid
4         0.4705         data/test set/quarters-060-01_c.mid
5         0.4371         data/test set/edgeroll-060-07_top.mid
6         0.4355         data/test set/alternating-060-02_baba-octup.mid
7         0.4244         data/test set/edgeroll-060-07_bottom.mid
8         0.3961         data/test set/alternating-060-02_cascas.mid
9         0.3825         data/test set/alternating-060-02_baba.mid
10        0.3818         data/test set/alternating-060-02_baba-tinyshift.mid
11        0.3805         data/test set/alternating-060-02_baba-microshift.mid
12        0.3729         data/test set/alt

**enter the index of the file you want to listen to:**

In [8]:
file_index = 1

In [9]:
# test file
MIDIPlayer(test_file, 300, styler=dark, title=os.path.basename(test_file))

In [10]:
# comparison file
comparison_file = sorted_sims[file_index][0]
MIDIPlayer(comparison_file, 300, styler=dark, title=os.path.basename(comparison_file))